In [ ]:
%matplotlib notebook
from IPython.display import clear_output
import pdb, time, math, numpy as np, gc, importlib, torch, os, cv2 as cv, ODEModel, matplotlib
import ot
import matplotlib.pyplot as plt
from torch import Tensor, nn
from torch.nn import functional as F 
from torch.autograd import Variable
from torch.distributions import MultivariateNormal
from torchdiffeq import odeint_adjoint as odeint 
from geomloss import SamplesLoss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import Utils, LearnVelTraj
importlib.reload(Utils)
from Utils import InputMapping, BoundingBox, ImageDataset, SaveTrajectory, ezshow, SaveTrajectory as st, MiscTransforms, MeshDataset, ezshow3D
importlib.reload(ODEModel)
from ODEModel import velocMLP, FfjordModel
importlib.reload(LearnVelTraj);
from LearnVelTraj import learn_vel_trajectory
from mpl_toolkits.mplot3d import axes3d   
import pandas as pd
import plotly 
import plotly.express as px
from plotly import tools
import plotly.graph_objs as go
from plotly.graph_objs import * #all the types of plots that we will plot here
# plotly.offline.init_notebook_mode()
# import wildmeshing as wm;
from meshpy.tet import MeshInfo, build


In [ ]:
# hand to cat: .01 jerk. .1 accel .1 rig
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_hand_cat/hand1.obj")
key2 = MeshDataset("scenes/hand_hand_cat/hand2.obj")
key3 = MeshDataset("scenes/hand_hand_cat/cat.obj")
meshArray = [key1,key2,key3]
# MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

reach=.5; 
model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_hand_cat_accel_reach/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True, reach=reach)
model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
render_reaches = [None]
for r in range(len(render_reaches)):
    render_reach = render_reaches[r]
    for j in range(16):
        # normalize mesharray
        meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 1, 2500)
        z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
        for i in range(len(meshArray)):
            meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
        st.save_trajectory(model, z_target_full, savedir=outfolder,
                                           savename="render_reach_"+str(render_reach)+"_"+str(j), nsteps=30, n=2500,
                                           dpiv=400, meshArray=meshArray, reach=render_reach)




In [ ]:
# hand to cat: .01 jerk. .1 accel .1 rig .1 reach
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_hand_cat/hand1.obj")
key2 = MeshDataset("scenes/hand_hand_cat/hand2.obj")
key3 = MeshDataset("scenes/hand_hand_cat/cat.obj")
meshArray = [key1,key2,key3]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_hand_cat_accel_less_reach/"
model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
# model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
for j in range(20):    
    # normalize mesharray
    meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 0, 2500)
    z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
    for i in range(len(meshArray)):
        meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
    st.save_trajectory(model, z_target_full, savedir=outfolder,
                                       savename="render"+str(j), nsteps=40, n=2500,
                                       dpiv=400, meshArray=meshArray)




In [ ]:
# hand to cat: .01 jerk. .1 accel .1 rig .5 reach
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_hand_cat/hand1.obj")
key2 = MeshDataset("scenes/hand_hand_cat/hand2.obj")
key3 = MeshDataset("scenes/hand_hand_cat/cat.obj")
meshArray = [key1,key2,key3]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_hand_cat_accel_reach/"
model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
# model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
for j in range(20):    
    # normalize mesharray
    meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 0, 2500)
    z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
    for i in range(len(meshArray)):
        meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
    st.save_trajectory(model, z_target_full, savedir=outfolder,
                                       savename="render"+str(j), nsteps=40, n=2500,
                                       dpiv=400, meshArray=meshArray)




In [ ]:
#rings: .01 jerk. .1 rig, .1 accel
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/rings/ring1.obj")
key2 = MeshDataset("scenes/rings/torus1.obj")
key3 = MeshDataset("scenes/rings/ring2.obj")
key4 = MeshDataset("scenes/rings/torus2.obj")
key5 = MeshDataset("scenes/rings/ring3.obj")
meshArray = [key1,key2,key3,key4,key5]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_rings_rigaccel/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=300, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
model.load_state(outfolder + 'models/state_final.tar')

for j in range(20):    
    # normalize mesharray
    meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 0, 2500)
    z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
    for i in range(len(meshArray)):
        meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
    st.save_trajectory(model, z_target_full, savedir=outfolder,
                                       savename="render"+str(j), nsteps=20, n=2500,
                                       dpiv=400, meshArray=meshArray)



In [ ]:
#rings: .01 jerk. 
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/rings/ring1.obj")
key2 = MeshDataset("scenes/rings/torus1.obj")
key3 = MeshDataset("scenes/rings/ring2.obj")
key4 = MeshDataset("scenes/rings/torus2.obj")
key5 = MeshDataset("scenes/rings/ring3.obj")
meshArray = [key1,key2,key3,key4,key5]
# MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_rings_base/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
model.load_state(outfolder + 'models/state_final.tar')

for j in range(20):    
    # normalize mesharray
    meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 0, 2500)
    z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
    for i in range(len(meshArray)):
        meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
    st.save_trajectory(model, z_target_full, savedir=outfolder,
                                       savename="render"+str(j), nsteps=20, n=2500,
                                       dpiv=400, meshArray=meshArray)




In [ ]:
#ringtorusring: .01 jerk. 
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/ring_torus_ring_turn/ring1.obj")
key2 = MeshDataset("scenes/ring_torus_ring_turn/torus.obj")
key3 = MeshDataset("scenes/ring_torus_ring_turn/ring2.obj")
meshArray = [key1,key2,key3]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_ringtorusring_base/"
model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
# model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
# meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 1, 10000)
# z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
# for i in range(len(meshArray)):
#     meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
# st.save_trajectory(model, z_target_full, savedir=outfolder,
#                                    savename="render", nsteps=20, n=10000,
#                                    dpiv=400, meshArray=meshArray)




In [ ]:
#sphere to cow to ring: .01 jerk. .2 accel
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/cow_sphere/sphere.obj")
key2 = MeshDataset("scenes/cow_sphere/cow.obj")
key3 = MeshDataset("scenes/cow_sphere/ring.obj")
meshArray = [key1,key2,key3]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_spherecowring_accel/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
model.load_state(outfolder + 'models/state_final.tar')

for j in range(20):    
    # normalize mesharray
    meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 0, 2500)
    z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
    for i in range(len(meshArray)):
        meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
    st.save_trajectory(model, z_target_full, savedir=outfolder,
                                       savename="render"+str(j), nsteps=20, n=2500,
                                       dpiv=400, meshArray=meshArray)




In [ ]:
#sphere to cow to ring: .01 jerk. 
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/cow_sphere/sphere.obj")
key2 = MeshDataset("scenes/cow_sphere/cow.obj")
key3 = MeshDataset("scenes/cow_sphere/ring.obj")
meshArray = [key1,key2,key3]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_spherecowring_base/"
model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 1, 10000)
z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
for i in range(len(meshArray)):
    meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
st.save_trajectory(model, z_target_full, savedir=outfolder,
                                   savename="render", nsteps=20, n=10000,
                                   dpiv=400, meshArray=meshArray)




In [ ]:
# hand to cat: .01 jerk. .3 accel .3 rig
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_hand_cat/hand1.obj")
key2 = MeshDataset("scenes/hand_hand_cat/hand2.obj")
key3 = MeshDataset("scenes/hand_hand_cat/cat.obj")
meshArray = [key1,key2,key3]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_hand_cat_more_accel/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 1, 10000)
z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
for i in range(len(meshArray)):
    meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
st.save_trajectory(model, z_target_full, savedir=outfolder,
                                   savename="render", nsteps=20, n=10000,
                                   dpiv=400, meshArray=meshArray)




In [ ]:
# hand to cat: .01 jerk. .1 accel .1 rig
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_hand_cat/hand1.obj")
key2 = MeshDataset("scenes/hand_hand_cat/hand2.obj")
key3 = MeshDataset("scenes/hand_hand_cat/cat.obj")
meshArray = [key1,key2,key3]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_hand_cat_accel/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
model.load_state(outfolder + 'models/state_final.tar')

for j in range(20):    
    # normalize mesharray
    meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 0, 2500)
    z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
    for i in range(len(meshArray)):
        meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
    st.save_trajectory(model, z_target_full, savedir=outfolder,
                                       savename="render"+str(j), nsteps=40, n=2500,
                                       dpiv=400, meshArray=meshArray)




In [ ]:
# hand to cat: .01 jerk.
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_hand_cat/hand1.obj")
key2 = MeshDataset("scenes/hand_hand_cat/hand2.obj")
key3 = MeshDataset("scenes/hand_hand_cat/cat.obj")
meshArray = [key1,key2,key3]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_hand_cat_base/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 1, 8000)
z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
for i in range(len(meshArray)):
    meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
st.save_trajectory(model, z_target_full, savedir=outfolder,
                                   savename="render2", nsteps=20, n=8000,
                                   dpiv=400, meshArray=meshArray)




In [ ]:
a,b = key1.sample();
a.shape
b.shape

In [ ]:
# hand to cat: .01 jerk. .1 curl target [0 -pi 0]. maybe the thumb will align to the tail?
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_cat/hand.obj")
key2 = MeshDataset("scenes/hand_cat/cat.obj")
meshArray = [key1,key2]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
# model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
# outfolder = "results/experiment_hand_to_cat_curly_pi/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)

model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_to_cat_curlz_pi/"
model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 1000, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)


# model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
# meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 1, 10000)
# z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
# for i in range(len(meshArray)):
#     meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
# st.save_trajectory(model, z_target_full, savedir=outfolder,
#                                    savename="render", nsteps=20, n=10000,
#                                    dpiv=400, meshArray=meshArray)




In [ ]:
# hand to cat: .001 rigid. result is basically not rigid at all. fit is pretty good though.
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_cat/hand.obj")
key2 = MeshDataset("scenes/hand_cat/cat.obj")
meshArray = [key1,key2]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=4, hidden_features=1024, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_to_cat_slight_rigid/"
model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 600, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
# model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
# meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 1, 10000)
# z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
# for i in range(len(meshArray)):
#     meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
# st.save_trajectory(model, z_target_full, savedir=outfolder,
#                                    savename="render", nsteps=20, n=10000,
#                                    dpiv=400, meshArray=meshArray)




In [ ]:
# hand to cat: .01 jerk. .1 rigid.
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_cat/hand.obj")
key2 = MeshDataset("scenes/hand_cat/cat.obj")
meshArray = [key1,key2]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_to_cat_rigid/"
model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=300, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
# model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
# meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 1, 10000)
# z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
# for i in range(len(meshArray)):
#     meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
# st.save_trajectory(model, z_target_full, savedir=outfolder,
#                                    savename="render", nsteps=20, n=10000,
#                                    dpiv=400, meshArray=meshArray)




In [ ]:
# hand to cat: .01 jerk.
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene
key1 = MeshDataset("scenes/hand_cat/hand.obj")
key2 = MeshDataset("scenes/hand_cat/cat.obj")
meshArray = [key1,key2]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
model = FfjordModel(sigmac = 3, n_freq = 100, hidden_layers=3, hidden_features=512, tdiv = 1, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_hand_to_cat_base/"
# model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 300, stepsperbatch = 50, n_subsample=300, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = False, lr = 2e-4, inner_percentage = .7, n_total = 10000, normalize=True)
model.load_state(outfolder + 'models/state_final.tar')

# normalize mesharray
meshSamplePoints = MeshDataset.meshArrayToPoints(meshArray, 1, 10000)
z_target_full, transform = ImageDataset.normalize_samples(torch.tensor(meshSamplePoints).to(device).float())
for i in range(len(meshArray)):
    meshArray[i].mesh.vertices = transform(torch.tensor(meshArray[i].mesh.vertices).to(device)).cpu().numpy()
st.save_trajectory(model, z_target_full, savedir=outfolder,
                                   savename="render", nsteps=20, n=10000,
                                   dpiv=400, meshArray=meshArray)


In [ ]:
# MeshDataset.meshArrayToTraces([MeshDataset("scenes/ring_torus_ring_turn/ring1.obj")], color=None, opacity=.5, show=True);

In [ ]:
# Load Scene: ring to torus to ring
key1 = MeshDataset("scenes/ring_torus_ring_turn/ring1.obj")
key2 = MeshDataset("scenes/ring_torus_ring_turn/torus.obj")
key3 = MeshDataset("scenes/ring_torus_ring_turn/ring2.obj")

meshArray = [key1,key2,key3]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=3, hidden_features=1024, tdiv = 2, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_ring_torus_ring/"
model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(keyMeshes=meshArray, n_iters = 2000, stepsperbatch = 200, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = True, lr = 2e-4,  inner_percentage = .6, n_total = 10000)

In [ ]:
import Utils, LearnVelTraj, ODEModel
importlib.reload(Utils); importlib.reload(ODEModel); importlib.reload(LearnVelTraj);
from ODEModel import velocMLP, FfjordModel; from LearnVelTraj import learn_vel_trajectory

# Load Scene: cow to lion
key1 = MeshDataset("scenes/cow_lion/cow.obj")
key2 = MeshDataset("scenes/cow_lion/lion.obj")
meshArray = [key1,key2]
MeshDataset.meshArrayToTraces(meshArray, color=None, opacity=.1, show=True);

# infer trajectory
model = FfjordModel(sigmac = 5, n_freq = 200, hidden_layers=3, hidden_features=1024, tdiv = 2, in_features=4, out_features=3, incrementalMask = True).to(device)
outfolder = "results/experiment_cow_to_lion/"
model.load_state(outfolder + 'models/state_final.tar')
model, losses, separate_losses, lrs, n_subs, separate_times = learn_vel_trajectory(meshArray, n_iters = 200, stepsperbatch = 50, n_subsample=1000, model=model, outname=outfolder, sqrtfitloss=True, detachTZM = True, lr = 2e-4, inner_percentage = .6, n_total = 10000)
# model.load_state(outfolder + 'models/state_0200.tar')

# st.save_trajectory(model, meshArray, savedir=outfolder,
#                                    savename="final", nsteps=20, n=800,
#                                    dpiv=400, meshArray=meshArray)

